In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f, Cylinder
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../functions.jl")

generate_q_init (generic function with 1 method)

### Creating the prismatic joints mechanisms

In [3]:
box_position = SVector(0.0, -0.065, 0.32)
box_dimensions = [0.03, 0.02, 0.03] 

m = Mechanism{Float64}("TestMechanism")

# add_frame!(m; id = "base_frame")
# add_joint!(m, Rigid(Transform(box_position));  parent = root_frame(m), child = "base_frame", id = "base_joint")
add_frame!(m; id="prism_frame_1")
add_joint!(m, Prismatic(SVector(1.0, 0.0, 0.0), Transform(box_position)); parent = root_frame(m), child="prism_frame_1", id="prism_joint_1")
add_frame!(m; id="prism_frame_2")
add_joint!(m, Prismatic(SVector(0.0, 1.0, 0.0)); parent="prism_frame_1", child="prism_frame_2", id="prism_joint_2")
add_frame!(m; id="ee frame")
add_joint!(m, Prismatic(SVector(0.0,0.0,1.0)); parent="prism_frame_2", child="ee frame", id="prism_joint_3")

add_coordinate!(m, FrameOrigin("ee frame"); id="ee position")
add_component!(m, PointMass(1.0, "ee position"); id="ee mass")

joint_damping = 50.0
add_coordinate!(m, JointSubspace("prism_joint_1"); id="prism joint 1")
add_component!(m, LinearDamper(joint_damping, "prism joint 1"); id="prism joint 1 damper")
add_coordinate!(m, JointSubspace("prism_joint_2"); id="prism joint 2")
add_component!(m, LinearDamper(joint_damping, "prism joint 2"); id="prism joint 2 damper")
add_coordinate!(m, JointSubspace("prism_joint_3"); id="prism joint 3")
add_component!(m, LinearDamper(joint_damping, "prism joint 3"); id="prism joint 3 damper")

# DEADZONE SPRINGS : CONSTRAINT THE MOTION INSIDE THE BOX

margin_factor = 0.8
add_deadzone_springs!(m, 1000.0, (-box_dimensions[1]*margin_factor, box_dimensions[1]*margin_factor), "prism joint 1")
add_deadzone_springs!(m, 1000.0, (-box_dimensions[2]*margin_factor, box_dimensions[2]*margin_factor), "prism joint 2")
add_deadzone_springs!(m, 1000.0, (-box_dimensions[3]*margin_factor, box_dimensions[3]*margin_factor), "prism joint 3")

add_gravity_compensation!(m, VMRobotControl.DEFAULT_GRAVITY)

add_coordinate!(m, ConstCoord(SVector(0.0, 0.0, 0.0)); id="base position")
add_coordinate!(m, CoordDifference("base position", "ee position"); id="ee position diff")
K_matrix = SMatrix{3, 3}(10., 0., 0., 0., 10., 0., 0., 0., 10.)
add_component!(m, LinearSpring(K_matrix,"ee position diff"); id="spring")

"spring"

In [ ]:
cm = compile(m)
kcache = Observable(new_kinematics_cache(cm))

# Setup the figure
fig1 = Figure(size=(700, 350))
ls1 = LScene(fig1[1, 1]; show_axis=false)

robotsketch!(ls1, kcache; scale=0.05, linewidth=2.5, transparency=true)

display(fig1)

GLMakie.Screen(...)

In [ ]:
q = zero_q(cm)
q̇ = zero_q̇(cm) 

dcache = new_dynamics_cache(cm)
g = VMRobotControl.DEFAULT_GRAVITY
prob = get_ode_problem(dcache, g, q, q̇, 10.0)
sol = solve(prob, Tsit5(), abstol=1e-3, reltol=1e-3)

display(fig1)
animate_robot_odesolution(fig1, sol, kcache, "robot_animation.mp4")

### Adjusting the position of the box w. r. t. the hand

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

In [9]:
box_position = SVector(0.06, -0.03, 0.37)
box_dimensions = [0.015, 0.05, 0.05] 

# Compile robot model
c_robot = compile(robot)
kcache = Observable(new_kinematics_cache(c_robot))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(c_robot, ls, "world")
# display_frame(c_robot, ls, "rh_fftip")
# display_frame(c_robot, ls, "rh_ffdistal")
# display_frame(c_robot, ls, "rh_ffmiddle")
# display_frame(c_robot, ls, "rh_ffproximal")
# display_frame(c_robot, ls, "rh_thtip")
# display_frame(c_robot, ls, "rh_thdistal")
# display_frame(c_robot, ls, "rh_thmiddle")


# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [box_position[1] - box_dimensions[1], box_position[2] + box_dimensions[2], box_position[3] - box_dimensions[3]]

t2 = [box_position[1] - box_dimensions[1], box_position[2] - box_dimensions[2], box_position[3] - box_dimensions[3]]

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]
T2 = [R t2; 0 0 0 1]
# display_transform(ls, T)
#display_transform(ls, T2)

box_visual_dimensions = box_dimensions - 0.004*SVector(1.,1.,1.)
box = Rect3f(Point3f(box_position) - Vec3f(box_visual_dimensions), 2*Vec3f(box_visual_dimensions))
mesh!(ls, box; color=:magenta, transparency=true)

# Display figure
display(fig)

GLMakie.Screen(...)